In [4]:
import sys
import os
import hydra
from config.config import Configuration
from autoencoder import AutoEncoder
from torch.utils.data import DataLoader, random_split
from torchvision import transforms, datasets
import pytorch_lightning as L


notebook_dir = os.path.dirname(os.path.abspath(os.getcwd()))
config_dir = os.path.join(notebook_dir,"tarea03-IA\\config")

sys.argv = [arg for arg in sys.argv if not arg.startswith("--")]

@hydra.main(config_path=config_dir, config_name="config", version_base=None)
def training_one(config: Configuration):
    transform = transforms.ToTensor()
    train_dataset = datasets.FashionMNIST(root=config.DATASET.DATA_DIR, train=True, transform=transform, download=True)
    test_dataset = datasets.FashionMNIST(root=config.DATASET.DATA_DIR, train=False, transform=transform, download=True)

    train_size = int(0.8 * len(train_dataset))
    val_size = len(train_dataset) - train_size

    train_data, val_data = random_split(train_dataset, [train_size, val_size])

    train_loader = DataLoader(train_data, batch_size=config.TRAIN.BATCH_SIZE, shuffle=True)
    val_loader = DataLoader(val_data, batch_size=config.TRAIN.BATCH_SIZE, shuffle=False)
    test_loader = DataLoader(test_dataset, batch_size=config.TRAIN.BATCH_SIZE, shuffle=False)

    model = AutoEncoder(config.MODEL.INPUT_SIZE, config.MODEL.LATENT_DIM, config.TRAIN.LEARNING_RATE)
    
    trainer = L.Trainer(max_epochs=config.TRAIN.NUM_EPOCHS, accelerator=config.TRAIN.ACCELERATION, 
                        precision=config.TRAIN.PRECISION)
    
    trainer.fit(model,train_loader,val_loader)
    trainer.test(model, test_loader)
    print(config)

@hydra.main(config_path=config_dir, config_name="config2", version_base=None)
def training_two(config: Configuration):
    print(config)

training_one()

e:\tarea03-IA\config
